<h2><u> Situation </u></h2>

Now I have a nice clean dataset with all entities resolved. However the meta-data is quite sparse. All I have is the source website, the ranking they gave (-1 if no ranking provided) and the title of the movie. Some preliminary data analysis follows here but first I'm going to augment the data using alternative data sources so that I can answer some more interesting questions.

In [3]:
import pandas as pd
from collections import Counter
#!pip install git+https://github.com/alberanid/imdbpy
#!pip install imdbpy
from imdb import IMDb
data = pd.read_csv("final.csv", index_col = 'Index')
pd.options.display.max_rows = 1000
ia = IMDb()

<h2><u> Task 1 </u></h2>

Let's augment the data with as many cool data sources as possible. The first step was to link all the movie titles I got with their IMDB ID. I used the IMDbpy API to do this. I got the docs off: https://readthedocs.org/projects/imdbpy/downloads/pdf/latest/



In [ ]:
'''
imdb_dict = dict((title, (None, None)) for title in data.Title)
for movie, _ in imdb_dict.items():
    imdb_movies = ia.search_movie(movie)
    for i in range(len(imdb_movies)):
        print(movie + " " + str(imdb_movies[i]) + " " + str(imdb_movies[i]['year']))
        choice = input("Correct title?: ")
        if choice == 'y':
            imdb_dict[movie] = (imdb_movies[i].movieID, imdb_movies[i]['year'])
            break

imdb_df = pd.DataFrame()
imdb_list = list()
for key, value in imdb_dict.items():
    movie = key
    imdb_id, year = value
    imdb_list.append([movie, imdb_id, year])
imdb_df = imdb_df.append(imdb_list)
imdb_df.columns = ['My_Title', 'IMDB_ID', 'Year']
imdb_df.head()
imdb_df.to_csv('imdb.csv')
'''

This was a nice exercise in fact checking all my data. I found a few inconsistencies where ampersands were dropped from the title like stan ollie -> stan & ollie. Now with the IMDB IDs I can pull out a wealth of information! 

In [15]:
ID = 1392190
movie = ia.get_movie(ID)
movie.infoset2keys
#not all features always found in infoset2keys!

{'main': ['original title',
  'cast',
  'genres',
  'runtimes',
  'countries',
  'country codes',
  'language codes',
  'color info',
  'aspect ratio',
  'sound mix',
  'box office',
  'certificates',
  'original air date',
  'rating',
  'votes',
  'cover url',
  'plot outline',
  'languages',
  'title',
  'year',
  'kind',
  'directors',
  'writers',
  'producers',
  'composers',
  'cinematographers',
  'editors',
  'editorial department',
  'casting directors',
  'production designers',
  'art directors',
  'set decorators',
  'costume designers',
  'make up department',
  'production managers',
  'assistant directors',
  'art department',
  'sound department',
  'special effects',
  'visual effects',
  'stunts',
  'camera department',
  'animation department',
  'casting department',
  'costume departmen',
  'location management',
  'music department',
  'script department',
  'transportation department',
  'miscellaneous',
  'thanks',
  'akas',
  'writer',
  'director',
  'top 250 

The features that I found interesring are selected in this features list:

In [16]:
features = ['title',
            'cast', 
            'genres',
            'runtimes',
            'box office',
            'rating',
            'votes',
            'kind',
            'directors',
            'writers',
            'producers',
            'composers',
            'cinematographers',
            'editors',
            'casting directors',
            'top 250 rank',
            'plot']

In [ ]:
#reading the list produced in first block
imdb_df = pd.read_csv('imdb.csv')

The way the API works is you send it the IMDB ID and it returns a set of keys that it has available. Not all movies have all the features in them, so I need to write a function that can check if the featuers exist in the set of keys.

In [19]:
for ID in [1392190]:
    movie = ia.get_movie(ID)
    for feature in features:
        if (feature in movie.infoset2keys['main']) or (feature in movie.infoset2keys['plot']):
            print(feature)
            print(movie[feature])
            print()
        else:
            print(feature + " not available\n")

title
Mad Max: Fury Road

cast
[<Person id:0362766[http] name:_Tom Hardy_>, <Person id:0000234[http] name:_Charlize Theron_>, <Person id:0396558[http] name:_Nicholas Hoult_>, <Person id:0117412[http] name:_Hugh Keays-Byrne_>, <Person id:2890541[http] name:_Josh Helman_>, <Person id:0428923[http] name:_Nathan Jones_>, <Person id:2368789[http] name:_Zoë Kravitz_>, <Person id:2492819[http] name:_Rosie Huntington-Whiteley_>, <Person id:2142336[http] name:_Riley Keough_>, <Person id:3880181[http] name:_Abbey Lee_>, <Person id:5196907[http] name:_Courtney Eaton_>, <Person id:0397398[http] name:_John Howard_>, <Person id:0141885[http] name:_Richard Carter_>, <Person id:5208473[http] name:_Iota_>, <Person id:0760151[http] name:_Angus Sampson_>, <Person id:0353228[http] name:_Jennifer Hagan_>, <Person id:0301885[http] name:_Megan Gale_>, <Person id:0415513[http] name:_Melissa Jaffer_>, <Person id:0432970[http] name:_Melita Jurisic_>, <Person id:0428143[http] name:_Gillian Jones_>, <Person id:08

Directors, actors etc are lists contain the Person object. Here's how you unpack the object. Upon examining this feature, I realized that the only really interesting information is the name of the actor themselves, so I'll just change the actor field to only have actor names in it.

In [20]:
movie['cast'][0].personID, movie['cast'][0]['name']

('0362766', 'Tom Hardy')

The box office value is not consistent so I'm not going to make use of it. I can try scraping this value later from Mojo. 

https://github.com/situkun123/Moive_mojo_project/blob/master/Moive_mojo_project.ipynb

For now, I'm focusing on adding the features I have. I'm going to use a nested dictionary to do this. Occasionally the API fails to fetch data and to catch that I used a try-except block. If any are found I can manually add them to the dictionary before I pickle it.

In [24]:
#to find
#imdb_dict_copy  = imdb_dict

In [40]:
def create_imdb_dict():
    imdb_dict = dict()
    err = []
    for i in range(len(imdb_df)):
        print("Collecting information for: " + imdb_df.loc[i].My_Title)
        print(str(i) + "/" + str(len(imdb_df)))
        ID = imdb_df.loc[i].IMDB_ID
        movie = ia.get_movie(ID)
        imdb_dict[ID] = dict()
        for feature in features:
            if (feature in movie.infoset2keys['main']) or (feature in movie.infoset2keys['plot']):
                try:
                    imdb_dict[ID][feature] = movie[feature]
                except:
                    err.append(ID)
    return imdb_dict, err
#imdb_dict, err = create_imdb_dict()

0/427
1/427
2/427
3/427
4/427
5/427
6/427
7/427
8/427
9/427
10/427
11/427
12/427
13/427
14/427
15/427
16/427
17/427
18/427
19/427
20/427
21/427
22/427
23/427
24/427
25/427
26/427
27/427
28/427
29/427
30/427
31/427
32/427
33/427
34/427
35/427
36/427
37/427
38/427
39/427
40/427
41/427
42/427
43/427
44/427
45/427
46/427
47/427
48/427
49/427
50/427
51/427
52/427
53/427
54/427
55/427
56/427
57/427
58/427
59/427
60/427
61/427
62/427
63/427
64/427
65/427
66/427
67/427
68/427
69/427
70/427
71/427
72/427
73/427
74/427
75/427
76/427
77/427
78/427
79/427
80/427
81/427
82/427
83/427
84/427
85/427
86/427
87/427
88/427
89/427
90/427
91/427
92/427
93/427
94/427
95/427
96/427
97/427
98/427
99/427
100/427
101/427
102/427
103/427
104/427
105/427
106/427
107/427
108/427
109/427
110/427
111/427
112/427
113/427
114/427
115/427
116/427
117/427
118/427
119/427
120/427
121/427
122/427
123/427
124/427
125/427
126/427
127/427
128/427
129/427
130/427
131/427
132/427
133/427
134/427
135/427
136/427
137/427
138/42

167/427
168/427
169/427
170/427
171/427
172/427
173/427
174/427
175/427
176/427
177/427
178/427
179/427
180/427
181/427
182/427
183/427
184/427
185/427
186/427
187/427
188/427
189/427
190/427
191/427
192/427
193/427
194/427
195/427
196/427
197/427
198/427
199/427
200/427
201/427
202/427
203/427
204/427
205/427
206/427
207/427
208/427
209/427
210/427
211/427
212/427
213/427
214/427
215/427
216/427
217/427
218/427
219/427
220/427
221/427
222/427
223/427
224/427
225/427
226/427
227/427
228/427
229/427
230/427
231/427
232/427
233/427
234/427
235/427
236/427
237/427
238/427
239/427
240/427
241/427
242/427
243/427
244/427
245/427
246/427
247/427
248/427
249/427
250/427
251/427
252/427
253/427
254/427
255/427
256/427
257/427
258/427
259/427
260/427
261/427
262/427
263/427
264/427
265/427
266/427
267/427
268/427
269/427
270/427
271/427
272/427
273/427
274/427
275/427
276/427
277/427
278/427
279/427
280/427
281/427
282/427
283/427
284/427
285/427
286/427
287/427
288/427
289/427
290/427
291/427


324/427
325/427
326/427
327/427
328/427
329/427
330/427
331/427
332/427
333/427
334/427
335/427
336/427
337/427
338/427
339/427
340/427
341/427
342/427
343/427
344/427
345/427
346/427
347/427
348/427
349/427
350/427
351/427
352/427
353/427
354/427
355/427
356/427
357/427
358/427
359/427
360/427
361/427
362/427
363/427
364/427
365/427
366/427
367/427
368/427
369/427
370/427
371/427
372/427
373/427
374/427
375/427
376/427
377/427
378/427
379/427
380/427
381/427
382/427
383/427
384/427
385/427
386/427
387/427
388/427
389/427
390/427
391/427
392/427
393/427
394/427
395/427
396/427
397/427
398/427
399/427
400/427
401/427
402/427
403/427
404/427
405/427
406/427
407/427
408/427
409/427
410/427
411/427
412/427
413/427
414/427
415/427
416/427
417/427
418/427
419/427
420/427
421/427
422/427
423/427
424/427
425/427
426/427


This process takes a fairly long time so I'm pickling the dictionary in case the notebook crashes.

In [42]:
import pickle
pickle.dump(imdb_dict , open("imdb.p", "wb"))
#imdb_dict = pickle.load(open( "imdb.p", "rb" ))

Above you can see what one item in this dictionary looks like. Let's now engineer some better features before we add everything to one large dataframe.

1. For actors, directors etc. I want to just have a list of names (not Person object) and maybe limit it say top 10 billed actors in the movie.

2. For plot I want to use NLP techniques to extract important keywords. I can then one hot encode this.

3. Finally one hot encode all list features.

4. Runtimes to be just 1 value from a list

The box office feature had inconsistent values as seen below. Some have the budget and gross, some just the opening weekend.


In [47]:
for key in imdb_dict.keys():
    if 'box office' in imdb_dict[key]:
        print(imdb_dict[key]['title'])
        print(imdb_dict[key]['box office'])
    else:
        break

Melancholia
{'Budget': '$7,400,000 (estimated)', 'Opening Weekend Denmark': 'DKK958,848, 29 May 2011'}
Mad Max: Fury Road
{'Budget': '$150,000,000 (estimated)', 'Cumulative Worldwide Gross': '$378,436,354'}
The Tree of Life
{'Budget': '$32,000,000 (estimated)', 'Opening Weekend United States': '$493,788, 30 May 2011', 'Cumulative Worldwide Gross': '$54,303,319, 27 Oct 2011'}
The Rider
{'Opening Weekend United States': '$42,244, 15 Apr 2018'}
A Separation
{'Budget': '$500,000 (estimated)', 'Opening Weekend Iran': '$100,000, 19 Mar 2011', 'Cumulative Worldwide Gross': '$24,426,169'}
Moonlight
{'Budget': '$1,500,000 (estimated)', 'Opening Weekend United States': '$1,488,740, 18 Nov 2016', 'Cumulative Worldwide Gross': '$55,561,162, 20 Mar 2017'}
The Fits
{'Opening Weekend United States': '$11,300, 05 Jun 2016'}
Margaret
{'Budget': '$14,000,000 (estimated)', 'Opening Weekend United States': '$7,525, 02 Oct 2011'}
Spider-Man: Into the Spider-Verse
{'Budget': '$90,000,000 (estimated)', 'Open

<h2><u> Task 2 </u></h2>
Preliminary data analysis

In [48]:
data.head()

,Title,Website,Rank
Index,,,
0,melancholia,vulture,1
1,mad max: fury road,vulture,2
2,the tree of life,vulture,3
3,the rider,vulture,4
4,a separation,vulture,5


<b> Question 1: </b> What were the top 10 movies mentioned across all the sources?

<b>Answer:</b>

In [49]:
#double square brackets to keep it as a dataframe
#columns option is required when using double braces
#have the column before the sum function in group_by
data.groupby('Title')[['Website']].count().nlargest(columns = 'Website', n = 10)

,Website
Title,
mad max: fury road,29
moonlight,28
get out,26
the social network,22
inside llewyn davis,19
boyhood,18
lady bird,18
roma,16
call me by your name,15


Okay but some of the data was ranked and some unranked, let's see what lists emerge from the ranked lists and the unranked lists. To split them up I use the split-apply-combine pattern of pandas.

<b> Question 2: </b> What were the top 10 movies mentioned across ranked and unranked sources?

<b>Answer:</b>

In [53]:
def is_rank(x):
    if x == -1: return 0
    else: return 1

data['is_ranked'] = data['Rank'].apply(lambda x: is_rank(x))
ranked_data = data[data['is_ranked'] == 1]
unranked_data = data[data['is_ranked'] == 0]
unranked_data.groupby('Title')[['Website']].count().nlargest(columns = 'Website', n = 10)

,Website
Title,
moonlight,9
get out,8
the social network,8
boyhood,7
lady bird,7
mad max: fury road,7
roma,7
black panther,6
her,5


In [54]:
ranked_data.groupby('Title')[['Website']].count().nlargest(columns = 'Website', n = 10)

,Website
Title,
mad max: fury road,22
moonlight,19
get out,18
inside llewyn davis,14
the social network,14
the master,13
the act of killing,12
the grand budapest hotel,12
boyhood,11


<b> Question 3: </b> Who were the top actors, directors, writers etc.?

<b>Answer:</b>

In [153]:
#function to get values of a certain key from a nested dictionary
def top_n_list(dic, person_key, n, top_billing = 0):
    people = []
    if top_billing == 0:
        for key in dic.keys():
            if person_key in dic[key]:
                for person in dic[key][person_key]:
                    try:
                        people.append((person.personID, person['name']))
                    except:
                        continue
            else:
                continue
    else:
        for key in dic.keys():
            if person_key in dic[key]:
                for person in dic[key][person_key][0 : top_billing]:
                    people.append((person.personID, person['name']))
            else:
                continue
    count_people = Counter(people)
    return count_people.most_common()[0:n]

In [154]:
x = top_n_list(imdb_dict, 'cast', 10)
x

[(('5241466', 'Mark Falvo'), 22),
 (('7419291', 'Arnold Montey'), 14),
 (('0171625', 'Bern Collaço'), 14),
 (('0498278', 'Stan Lee'), 13),
 (('3485845', 'Adam Driver'), 11),
 (('0424060', 'Scarlett Johansson'), 10),
 (('5857646', 'Joseph Oliveira'), 10),
 (('0842770', 'Tilda Swinton'), 10),
 (('6768665', 'Patti Schellhaas'), 10),
 (('0000168', 'Samuel L. Jackson'), 10)]

The top 4 actors are all extras actors! Luckily the cast is sorted by their billing order so I can use the top_n feature of my function to just select the top 20 billed actors from each movie which I think is a fair way to find the top actors.

<h3> Top actors of the decade</h3>

In [155]:
x = top_n_list(imdb_dict, 'cast', n = 10, top_billing = 20)
x

[(('0424060', 'Scarlett Johansson'), 10),
 (('3485845', 'Adam Driver'), 10),
 (('1209966', 'Oscar Isaac'), 8),
 (('0000168', 'Samuel L. Jackson'), 8),
 (('0842770', 'Tilda Swinton'), 8),
 (('0749263', 'Mark Ruffalo'), 7),
 (('1727304', 'Domhnall Gleeson'), 7),
 (('0331516', 'Ryan Gosling'), 7),
 (('1256532', 'Jon Bernthal'), 7),
 (('0262635', 'Chris Evans'), 7)]

There we go! The critic favorites of the decade!

<h3> Top directors of the decade</h3>

In [157]:
x = top_n_list(imdb_dict, 'directors', n = 10)
x

[(('0898288', 'Denis Villeneuve'), 5),
 (('0634240', 'Christopher Nolan'), 4),
 (('0169806', 'Taika Waititi'), 4),
 (('0751577', 'Anthony Russo'), 4),
 (('0751648', 'Joe Russo'), 4),
 (('0000233', 'Quentin Tarantino'), 3),
 (('0000759', 'Paul Thomas Anderson'), 3),
 (('0000229', 'Steven Spielberg'), 3),
 (('0487166', 'Yorgos Lanthimos'), 3),
 (('1509478', 'Benny Safdie'), 3)]

<h3> Top producers of the decade</h3>

In [158]:
x = top_n_list(imdb_dict, 'producers', n = 10)
x

[(('0748784', 'Scott Rudin'), 16),
 (('0498278', 'Stan Lee'), 16),
 (('5398118', 'Olivier Père'), 16),
 (('2691892', 'Megan Ellison'), 13),
 (('0000093', 'Brad Pitt'), 12),
 (('0022285', 'Victoria Alonso'), 12),
 (('0195669', "Louis D'Esposito"), 12),
 (('0270559', 'Kevin Feige'), 12),
 (('0743882', 'Tessa Ross'), 11),
 (('4791912', 'Eli Bush'), 11)]

<h3> Top composers of the decade</h3>

In [159]:
x = top_n_list(imdb_dict, 'composers', n = 10)
x

[(('0006035', 'Alexandre Desplat'), 10),
 (('0315974', 'Michael Giacchino'), 10),
 (('0001877', 'Hans Zimmer'), 8),
 (('0002354', 'John Williams'), 5),
 (('0001937', 'Marco Beltrami'), 5),
 (('0339351', 'Jonny Greenwood'), 4),
 (('0002353', 'Thomas Newman'), 4),
 (('0001980', 'Carter Burwell'), 4),
 (('2273444', 'Henry Jackman'), 4),
 (('0510533', 'Giong Lim'), 4)]

<h3> Top cinematographers of the decade</h3>

In [160]:
x = top_n_list(imdb_dict, 'cinematographers', n = 10)
x

[(('0005683', 'Roger Deakins'), 6),
 (('0523881', 'Emmanuel Lubezki'), 4),
 (('0393240', 'Kyung-pyo Hong'), 4),
 (('0494617', 'Yorick Le Saux'), 4),
 (('0568174', 'Michael McDonough'), 4),
 (('0887227', 'Hoyte Van Hoytema'), 4),
 (('0451787', 'Darius Khondji'), 4),
 (('1831620', 'Sean Porter'), 4),
 (('0006509', 'Rodrigo Prieto'), 4),
 (('1023204', 'Ben Davis'), 4)]

<h3> Top editors of the decade</h3>

In [161]:
x = top_n_list(imdb_dict, 'editors', n = 10)
x

[(('0907863', 'Joe Walker'), 6),
 (('0328557', 'Affonso Gonçalves'), 5),
 (('0809059', 'Lee Smith'), 5),
 (('0285701', 'Jeffrey Ford'), 5),
 (('0711235', 'Fred Raskin'), 4),
 (('0561430', 'Yorgos Mavropsaridis'), 4),
 (('0918733', 'Andrew Weisblum'), 4),
 (('2352780', 'Jennifer Lame'), 4),
 (('0773113', 'Matthew Schmidt'), 4),
 (('1477623', 'Nat Sanders'), 3)]

<h3> Top writers of the decade</h3>

In [162]:
x = top_n_list(imdb_dict, 'writers', n = 10)
x

[(('0456158', 'Jack Kirby'), 13),
 (('0498278', 'Stan Lee'), 11),
 (('0094435', 'Bong Joon Ho'), 5),
 (('0634240', 'Christopher Nolan'), 5),
 (('1921680', 'Steve Englehart'), 5),
 (('0004056', 'Andrew Stanton'), 5),
 (('0800209', 'Joe Simon'), 5),
 (('0027572', 'Wes Anderson'), 5),
 (('0520488', 'Phil Lord'), 4),
 (('4401003', 'Derek Kolstad'), 4)]

<h3> Top genres of the decade</h3>

In [138]:
imdb_dict[1392190]['genres']

['Action', 'Adventure', 'Sci-Fi', 'Thriller']

In [164]:
genres = []
for key in imdb_dict.keys():
    for genre in imdb_dict[key]['genres']:
        genres.append(genre)
count_people = Counter(genres)
count_people.most_common()[0:10]

[('Drama', 299),
 ('Comedy', 98),
 ('Thriller', 93),
 ('Adventure', 63),
 ('Crime', 63),
 ('Action', 60),
 ('Documentary', 59),
 ('Romance', 58),
 ('Sci-Fi', 54),
 ('Biography', 49)]

Dramas just dominate the film market. I'm quite surprised that comedies was the second most repeated genre, just because of how badly pure comedy movies tend to get rated on IMDB and Rotten Tomatoes.